# Analysis

In [ ]:
%load_ext autoreload
%autoreload 2
%aimport theme
import pandas as pd
import altair as alt
import pygwalker as pyg
from altair import datum
from theme import apply_theme
alt.data_transformers.disable_max_rows(); # Allow using rows more than 5000

## Set Const Variables

In [ ]:
TIME_STAMP_FOLDER = 'JAN-10-2024'
A11Y_CATEGORIES = [
    ('d', 'data-portal', 'Data Portals'), #, 'short_name', 'url'),
    ('j', 'journal-portal', 'Journal Portals'), # 'title', 'url'),
    ('g', 'gov', "US Government Websites"), # 'name', 'name')
]
AGG_CATEGORIES = ['', '_agg', '_agg_alt']
TOP_CNT = 10 # for printing top websites

## Accessibility Issues

In [ ]:
issues_df = pd.DataFrame()
for (id, category, title) in A11Y_CATEGORIES:
    _ = pd.read_csv(
        f'../javascript/{TIME_STAMP_FOLDER}/{category}_a11y_issues.csv',
        header=None,
        names=['id', 'impact', 'description']
    )
    issues_df = pd.concat([issues_df, _])
issues_df.drop_duplicates(inplace=True)
issues_df.to_csv(f'../javascript/{TIME_STAMP_FOLDER}/all_a11y_issues.csv', index=False)
issues_df

In [ ]:
issues_df[issues_df.impact == 'serious']

In [ ]:
ISSUE_IDS = issues_df.id.tolist()

In [ ]:
issues_df[issues_df.description.str.contains('alternate')]
ALT_ISSUE_IDS = issues_df[issues_df.description.str.contains('alternate')].id.tolist()
ALT_ISSUE_IDS

## Load Accessibility Evaluation Results

In [ ]:
df = {}

for (id, category, title) in A11Y_CATEGORIES:
    df[id] = pd.read_csv(
        f'../javascript/{TIME_STAMP_FOLDER}/{category}_a11y_results.csv',
        header=None,
        names=['page_id', 'issue_id', 'violations', 'passes']
    )

df['d']

### Merge additional datasets we selectively collected

#### NEI

In [ ]:
nei = pd.read_csv(
    f'../javascript/{TIME_STAMP_FOLDER}/nei-data-portal_a11y_results.csv',
    header=None,
    names=['page_id', 'issue_id', 'violations', 'passes']
)
df['d'] = pd.concat([df['d'], nei])
df['d']

#### NIH

In [ ]:
nih = pd.read_csv(
    f'../javascript/{TIME_STAMP_FOLDER}/nih-data-portal_a11y_results.csv',
    header=None,
    names=['page_id', 'issue_id', 'violations', 'passes']
)
df['d'] = pd.concat([df['d'], nih])
df['d']

### Harvard

In [ ]:
harvard = pd.read_csv(
    f'../javascript/{TIME_STAMP_FOLDER}/harvard-data-portal_a11y_results.csv',
    header=None,
    names=['page_id', 'issue_id', 'violations', 'passes']
)
df['d'] = pd.concat([df['d'], harvard])
df['d']

## Calculate Accessibility Measurement (Failure Rate or FR)

In [ ]:
for (id, category, title) in A11Y_CATEGORIES:
    _ = df[id].copy()
    
    # To group by page_id, uncomment the following lines
    # _df.drop(['issue_id'], axis=1, inplace=True)
    # _df = _df.groupby(['page_id']).sum().reset_index()
    
    _['total_checks'] = _['violations'] + _['passes']
    _['failure_rate'] = _['violations'] / _['total_checks']
    df[id] = _
df['d'].head(5)

## Aggregation By Page ID

In [ ]:
for (id, category, title) in A11Y_CATEGORIES:
    _ = df[id].copy()
    
    # CHECKING...
    # _ = _[_.issue_id != 'region']

    _.drop(['issue_id'], axis=1, inplace=True)
    _ = _.groupby(['page_id']).sum().reset_index()
    _['failure_rate'] = _['violations'] / _['total_checks']
    df[id + '_agg'] = _
df['d_agg'].head(5)

## Aggregation w/Alt-related Issues Only

In [ ]:
for (id, category, title) in A11Y_CATEGORIES:
    _ = df[id].copy()
    _ = _[_['issue_id'].isin(ALT_ISSUE_IDS)]
    _.drop(['issue_id'], axis=1, inplace=True)
    _ = _.groupby(['page_id']).sum().reset_index()
    _['failure_rate'] = _['violations'] / _['total_checks']
    df[id + '_agg_alt'] = _
df['d_agg_alt'].head(5)

## Merge Metadata

### For Data Portals and Journal Websites

In [ ]:
for (id, category, title) in A11Y_CATEGORIES:
    if(category == 'gov'):
        continue

    for ver in ['', '_agg', '_agg_alt']:
        _ = df[id + ver].copy()
        _['id'] = _['page_id'].apply(lambda x: x.split('_')[0])
        meta = pd.read_csv(f'../output/Nov-21-2023/{category}_metadata.csv')

        _['id'] = _['id'].astype(str)
        meta['id'] = meta['id'].astype(str)

        _ = _.merge(meta, left_on='id', right_on='id', how='left')

        # Some data cleaning
        if id == 'd':
            # Group NIH 
            NIH_INSTS = [
                'National Center for Biotechnology Information',
                'National Cancer Institute',
                'National Heart, Lung, and Blood Institute',
                'National Center for Advancing Translational Sciences',
                'National Institutes of Health',
                'National Human Genome Research Institute',
                'National Institute of Environmental Health Sciences',
                'National Library of Medicine',
                'National Institute of Standards and Technology',
                'National Institute of Health',
                'National Institute on Aging',
                'National Institute of Neurological Disorders & Stroke',
                'National Institute of Child Health and Human Development',
                'National Eye Institute', # none found
                'National Institute of Allergy and Infectious Diseases',
                'National Institute of Arthritis and Musculoskeletal and Skin Diseases'
            ]
            # _.loc[_.host_institution.isin(NIH_INSTS), 'host_institution'] = 'National Institutes of Health'
        # elif id == 'jp':
            # _.loc[_.publisher.str.contains('Elsevier') == True, 'publisher'] = 'Elsevier'
            # _.loc[_.publisher.str.contains('Springer') == True, 'publisher'] = 'Springer-related'

        df[id + ver] = _
df['j'].head(5)

### Add page ypes

In [ ]:
for (id, category, title) in A11Y_CATEGORIES:
    for ver in ['', '_agg', '_agg_alt']:
        df[id + ver]['page_type'] = df[id + ver].page_id.apply(lambda x: x.split('_')[1] if '_' in str(x) else 'home')
df['d_agg'].head(5)

### Load Metadata for Gov

In [ ]:
for var in AGG_CATEGORIES:
    gov_meta = pd.read_csv(f'../javascript/{TIME_STAMP_FOLDER}/gov_pages.csv', sep=',', header=None, names=['name', 'type', 'inst', 'desc', 'city', 'state', 'blank'])
    gov_meta.reset_index(inplace=True)
    df['g' + var] = df['g' + var].merge(gov_meta, left_on='page_id', right_on='index', how='left')
    df['g' + var]['url'] = df['g' + var]['name']

### Load Metadata for NEI

In [ ]:
for var in AGG_CATEGORIES:
    _meta = pd.read_csv(f'../javascript/{TIME_STAMP_FOLDER}/nei-data-portal_pages.csv')
    _meta.set_index('page_id', inplace=True)
    
    _ = df['d' + var].copy()
    _.set_index('page_id', inplace=True)
    _[_.isnull()] = _meta
    _.reset_index(inplace=True)
    df['d' + var] = _

### Load Metadata for NIH

In [ ]:
for var in AGG_CATEGORIES:
    _meta = pd.read_csv(f'../javascript/{TIME_STAMP_FOLDER}/nih-data-portal_pages.csv')
    _meta['short_name'] = _meta.Repository_Name
    _meta['host_institution'] = 'National Institutes of Health'
    _meta['country'] = 'United States'
    _meta.set_index('page_id', inplace=True)

    _ = df['d' + var].copy()
    _.set_index('page_id', inplace=True)
    _[_.isnull()] = _meta
    _.reset_index(inplace=True)
    df['d' + var] = _

### Load Metadata for Harvard

In [ ]:
for var in AGG_CATEGORIES:
    _meta = pd.read_csv(f'../javascript/{TIME_STAMP_FOLDER}/harvard-data-portal_pages.csv')
    _meta.set_index('page_id', inplace=True)
    
    _ = df['d' + var].copy()
    _.set_index('page_id', inplace=True)
    _[_.isnull()] = _meta
    _.reset_index(inplace=True)
    df['d' + var] = _

In [ ]:
df['d_agg'][df['d_agg'].page_id.str.contains('harvard')]

## Data Cleaning

In [ ]:
# df['j_agg'][df['j_agg'].title == 'Nature Communications']

### Column Names

In [ ]:
for var in AGG_CATEGORIES:
    df['d' + var]['title'] = df['d' + var]['short_name']
    df['g' + var]['title'] = df['g' + var]['name']

### Country Names

In [ ]:
def rename_countries(x: str):
    if x == 'Korea Republic of' or x == 'Korea, Republic of':
        return 'South Korea'
    elif x == 'Korea, Democratic People"S Republic of' or x == 'Korea, Democratic People':
        return 'North Korea'
    elif x == 'Russian Federation':
        return 'Russia'
    elif x == 'Iran, Islamic Republic Of':
        return 'Iran'
    else:
        return x
    
for var in AGG_CATEGORIES:
    df['d' + var].country = df['d' + var].country.apply(lambda x: rename_countries(x))
    df['j' + var].country = df['j' + var].country.apply(lambda x: rename_countries(x))

# Print
# countries = [x for x in list(set(df['d'].country.tolist() + df['j'].country.unique().tolist())) if str(x) != 'nan']
# countries.sort()
# countries

### Add Continent

In [ ]:
continent_country_map = pd.read_csv('https://raw.githubusercontent.com/dbouquin/IS_608/master/NanosatDB_munging/Countries-Continents.csv')
continent_country_map = continent_country_map.rename(columns={
    'Country': 'country',
    'Continent': 'continent'
})

def clean_country_names(x):
    if x == 'US':
        return 'United States'
    elif x == 'Korea, South':
        return 'South Korea' 
    elif x == 'Korea, North':
        return 'North Korea'
    elif x == 'Russian Federation':
        return 'Russia'
    elif x == 'Samoa':
        return 'American Samoa'
    elif x == 'Vietnam':
        return 'Viet Nam'
    elif x == 'Serbia':
        return 'Serbia and Montenegro'
    else:
        return x

continent_country_map.country = continent_country_map.country.apply(lambda x: clean_country_names(x))

for id in ['d', 'j']:
    for var in AGG_CATEGORIES:
        _ = df[id + var].copy()
        _ = _.merge(continent_country_map, left_on='country', right_on='country', how='left')
        _.loc[_.continent.isnull(), 'continent'] = _[_.continent.isnull()].country.apply(lambda x: 'Europe' if x == 'Czech Republic' or x == 'Guadeloupe' else x)

        # Some manual correction
        _.continent = _.continent.apply(lambda x: 'Europe' if x == 'Serbia' else 'Asia' if x == 'Taiwan' or x == 'Hong Kong' or x == 'Brunei Darussalam' else 'North America' if x == 'Puerto Rico' else x)
        
        df[id + var] = _
# _[_.continent.isnull()].country.unique().tolist()
_.head(5)

### Journal Publisher Names

In [ ]:
_ = pd.read_csv(f'../javascript/{TIME_STAMP_FOLDER}/Publishers of Journal Portals - Sheet1.csv')
_.Cleaned.fillna(_.Original, inplace=True)
mapping = dict(zip(_.Original, _.Cleaned))

for var in AGG_CATEGORIES:
    df['j' + var].publisher = df['j' + var].publisher.apply(lambda x: mapping[x] if x in mapping else x)
df['j'].head(5)

### Journal Filtering

In [ ]:
_ = pd.read_csv(f'../javascript/{TIME_STAMP_FOLDER}/Filtering of Journals - Sheet1.csv')
_.rename(columns={'If filter "v", otherwise empty': 'is_filter'}, inplace=True)
_.is_filter = _.is_filter.apply(lambda x: True if x == 'v' else False)
mapping = dict(zip(_.Title, _.is_filter))

for var in AGG_CATEGORIES:
    df['j' + var]['is_filter'] = df['j' + var].title.apply(lambda x: mapping[x] if x in mapping else False)
    df['j' + var] = df['j' + var][df['j' + var].is_filter == False]
df['j'].head(5)

### More Journal Filtering Based on "Areas"

In [ ]:
# What are the unique areas?
combined_list = [area.split('; ') for area in df['j'].areas.unique().tolist()]
flat_list = [item for sublist in combined_list for item in sublist]
flat_list = list(set(flat_list))
flat_list.sort()
# flat_list

In [ ]:
keep = [
    'Agricultural and Biological Sciences',
    'Biochemistry, Genetics and Molecular Biology',
    'Dentistry',
    'Health Professions',
    'Immunology and Microbiology',
    'Medicine',
    'Multidisciplinary',
    'Neuroscience',
    'Nursing',
    'Pharmacology, Toxicology and Pharmaceutics',
    'Psychology'
]
def isKeeping(areas: str):
    if areas == 'Multidisciplinary' or any([k in areas for k in keep]):
        return True
    return False

for var in AGG_CATEGORIES:    
    df['j' + var]['is_keep'] = df['j' + var].areas.apply(lambda x: isKeeping(x))
    df['j' + var] = df['j' + var][df['j' + var].is_keep == True]
    df['j' + var].drop(['is_keep'], axis=1, inplace=True)
    df['j' + var]

In [ ]:
# df['j_agg'][df['j_agg'].publisher.str.contains('Massachussetts Medical Society')]

## Export Data for Collaboration

In [ ]:
from datetime import date
from pathlib import Path

for key in ['j_agg', 'd_agg', 'g_agg']:
    file_name = 'journal-websites' if key == 'j_agg' else 'data-portals' if key == 'd_agg' else 'us-goverment-websites'
    today = date.today()
    folder = f'../output/share/{today}'
    Path(folder).mkdir(parents=True, exist_ok=True)
    df[key].to_csv(f'{folder}/{file_name}_{today}.csv', index=False)

## Descriptive Statistics

### # of Pages

In [ ]:
GOV_NUM_PAGES = len(df['g'].page_id.unique().tolist())
DP_NUM_PAGES = len(df['d'].page_id.unique().tolist())
JW_NUM_PAGES = len(df['j'].page_id.unique().tolist())

GOV_NUM_PAGES, DP_NUM_PAGES, JW_NUM_PAGES

### # of Unique Websites

In [ ]:
GOV_NUM_WEBSITES = len(df['g'].page_id.apply(lambda x: x.split('_')[0] if '_' in str(x) else x).unique().tolist())
DP_NUM_WEBSITES = len(df['d'].page_id.apply(lambda x: x.split('_')[0] if '_' in str(x) else x).unique().tolist())
JW_NUM_WEBSITES = len(df['j'].page_id.apply(lambda x: x.split('_')[0] if '_' in str(x) else x).unique().tolist())

GOV_NUM_WEBSITES, DP_NUM_WEBSITES, JW_NUM_WEBSITES

In [ ]:
# # Columns with list of values
# COLUMNS_WITH_LIST = list(filter(lambda x: 'list' in x, df['d_agg_alt'].columns))
# df['d_agg'][['title'] + COLUMNS_WITH_LIST].head(10)

In [ ]:
# # Columns with list of values
# df['j_agg'][['title', 'categories', 'areas']].head(10)

### # of Unique Journal Areas

In [ ]:
# Unique categories
df['j'].areas.str.split('; ', expand=True).stack().unique().tolist()

In [ ]:
# Unique categories
# _ = df['d'].copy()
# _[_.host_institution == 'National Eye Institute']

### # of Unique Countries

In [ ]:
# Numb of unique countries
len(
    [x for x in list(set(df['d'].country.tolist() + df['j'].country.unique().tolist())) if str(x) != 'nan']
)

### # of Journal Publishrs

In [ ]:
# Numb of jorunal publishers
len(df['j'].publisher.unique().tolist())

### # of Data Portals' Host Institution

In [ ]:
# Numb of data portal institutions
len(df['d'].host_institution.unique().tolist())

### Mean FRs

In [ ]:
US_GOV_FR_MEAN = df['g_agg'].failure_rate.mean()
US_GOV_FR_MEAN, df['g_agg'].failure_rate.min(), df['g_agg'].failure_rate.max()

In [ ]:
DP_FR_MEAN = df['d_agg'].failure_rate.mean()
DP_FR_MEAN, df['d_agg'].failure_rate.max(), df['d_agg'].failure_rate.min()

In [ ]:
JW_FR_MEAN = df['j_agg'].failure_rate.mean()
JW_FR_MEAN, df['j_agg'].failure_rate.min(), df['j_agg'].failure_rate.max()

In [ ]:
_ = pd.concat([df['g_agg'], df['d_agg'], df['j_agg']])
_.failure_rate.mean()

In [ ]:
for (id, category, title) in A11Y_CATEGORIES:
    print(
        id, 
        df[id + '_agg'].violations.mean(),
        df[id + '_agg'].violations.min(),
        df[id + '_agg'].violations.max()
    )

In [ ]:
for (id, category, title) in A11Y_CATEGORIES:
    print(
        id, 
        df[id + '_agg'].total_checks.mean(),
        df[id + '_agg'].total_checks.min(),
        df[id + '_agg'].total_checks.max()
    )

# Visualize

In [ ]:
walker = pyg.walk(df['d_agg'])

## How does the distribution looks like?

In [ ]:
plot = None
for (id, category, title) in A11Y_CATEGORIES:
    # bests = df[id + '_agg'].sort_values(by='failure_rate', ascending=True).head(TOP_CNT)[name_field].tolist()
    # worsts = df[id + '_agg'].sort_values(by='failure_rate', ascending=False).head(TOP_CNT)[name_field].tolist()
    # print(f"{title}' Best:")
    # print(f"\t{', '.join(bests)}")
    # print(f"{title}' Worst:")
    # print(f"\t{', '.join(worsts)}")
    _ = (
        alt.Chart(
            df[id + '_agg'][df[id + '_agg'].failure_rate > 0]
        ).mark_bar(
            # opacity=0.01
            color='#56B4E9'
        ).encode(
            alt.X(f'failure_rate:Q', title='Failure Rate').bin(extent=[0, 1], step=0.01).scale(domain=[0, 0.5], clamp=True).axis(format='%'),
            alt.Y('count()', title="The Number of Websites").scale(type='linear'),
        ).properties(
            title={
                "text": title,
                "color": "grey"
            },
            height=300,
            width=500
        )
    )

    baseline = (
        _.mark_rule(
            color='black',
            size=2,
            # size=500 / len(COUNTRY_SORT),
            strokeDash=[4, 2]
        ).encode(
            alt.X(f'baseline:Q', title='Failure Rate'),
            y=alt.Y()
        ).transform_calculate(
            baseline=f"{US_GOV_FR_MEAN}"
        )
    )
    
    _ = _ + baseline

    plot = _ if plot is None else plot | _

plot = plot.properties(
    title={
        'text': 'The Distribution of Failure Rate of Webpages',
        'subtitle': '* Dashed line represents the average failure rate of US government websites',
        'subtitleColor': 'grey'
    }
)

apply_theme(plot)

## Which are at the top and the bottom?

In [ ]:
CUT = 10
for (id, category, title) in A11Y_CATEGORIES:
    print(title)
    print('Bottom ' + str(CUT))
    print(df[id + '_agg'].sort_values(by=['failure_rate'], ascending=False).head(CUT).url.tolist())
    print('Top ' + str(CUT))
    print(df[id + '_agg'].sort_values(by=['failure_rate'], ascending=True).head(CUT).url.tolist())

    # This shows that we really need to filter data properly!
    # There are many 404 pages in the data

## What about the # of DOM elements?

In [ ]:
plot = None
for (id, category, title) in A11Y_CATEGORIES:
    # bests = df[id + '_agg'].sort_values(by='total_checks', ascending=True).head(TOP_CNT)[name_field].tolist()
    # worsts = df[id + '_agg'].sort_values(by='total_checks', ascending=False).head(TOP_CNT)[name_field].tolist()
    # print(f"{title}' Lowest:")
    # print(f"\t{', '.join(bests)}")
    # print(f"{title}' Highest:")
    # print(f"\t{', '.join(worsts)}")
    _ = (
        alt.Chart(
            df[id + '_agg']
        ).mark_bar(
            # opacity=0.3,
            color='#56B4E9',
            # stroke='black'
        ).encode(
            alt.X(f'total_checks:Q', title='The Number of DOM Elements').bin(extent=[0, 10000], step=100).scale(padding=0, type='linear', domain=[0, 3200], clamp=True),
            alt.Y('count()', title='The Number of Websites').scale(type='linear'),
        ).properties(
            title={
                "text": title,
                "color": "grey"
            },
            height=300,
            width=500
        )
    )
    
    plot = _ if plot is None else plot | _

plot = plot.properties(
    title='The Distribution of the Number of DOM Elements'
)

apply_theme(plot)

## Is there a correlation between the webpage complexity and failure rate?

In [ ]:
plot = None
for (id, category, title) in A11Y_CATEGORIES:
    _ = (
        alt.Chart(
            df[id + '_agg']
        ).mark_point(
            filled=True,
            opacity=0.3,
            color='#56B4E9'
        ).encode(
            alt.X(f'total_checks:Q', title='The Number of DOM Elements').scale(type='log'),
            alt.Y('failure_rate:Q', title='Failure Rate').scale(domain=[0, 1]),
            alt.Tooltip([f'title:N', f'url:N'])
        ).properties(
            title={
                'text': title,
                'color': 'grey'
            },
            width=400,
            height=300
        )
    )
    
    nei = _.transform_filter(datum.host_institution == 'National Eye Institute').mark_point(
        opacity=1,
        filled=True,
        color='#D55D00'    
    )

    nih = _.transform_filter(datum.host_institution == 'National Institutes of Health').mark_point(
        opacity=1,
        filled=True,
        color='#009E73'    
    )

    baseline = (
        _.mark_rule(
            color='black',
            size=2,
            # size=500 / len(COUNTRY_SORT),
            strokeDash=[4, 2]
        ).encode(
            alt.Y(f'baseline:Q', title='Failure Rate'),
            alt.X()
        ).transform_calculate(
            baseline=f"{US_GOV_FR_MEAN}"
        )
    )
    
    _ = _ + nih + nei + baseline

    plot = _ if plot is None else plot | _

plot = plot.properties(
    title={
        'text': 'The Number of Items Checked vs. Failure Rate',
        'subtitle': '* Dashed line represents the average failure rate of US government websites',
        'subtitleColor': 'grey'
    }
)

apply_theme(plot)

## How about the relation with impact score?

In [ ]:
plot = None
for (id, category, title) in A11Y_CATEGORIES:
    if id == 'g':
        continue
    
    impact_field = 'h_index' if id == 'j' else 'citation'

    _ = (
        alt.Chart(
            df[id + '_agg']
        ).mark_point(
            # filled=True,
            opacity=0.3,
            color='#56B4E9'
        ).encode(
            alt.X(f'{impact_field}:Q', title=impact_field.capitalize()),#.scale(type='symlog'),
            alt.Y('failure_rate:Q', title='Failure Rate').scale(domain=[0, 1]),
            alt.Tooltip([f'title:N', f'url:N'])
        ).properties(
            title={
                'text': title,
                'color': 'grey'
            },
            width=400,
            height=300
        )
    )

    t = (
        alt.Chart(
            df[id + '_agg'][df[id + '_agg'][impact_field] > (10000 if id == 'dp' else 800)]
        ).mark_text(
            align='left',
            limit=200,
            opacity=0.5,
            fontSize=14,
            dx=10,
            baseline='bottom'
        ).encode(
            alt.X(f'{impact_field}:Q', title=impact_field.capitalize()),#.scale(type='log'),
            alt.Y('failure_rate:Q', title='Failure Rate').scale(domain=[0, 1]),
            alt.Text(f'title:N'),
            alt.Tooltip([f'title:N', f'url:N'])
        )
    )

    baseline = (
        _.mark_rule(
            color='black',
            size=2,
            # size=500 / len(COUNTRY_SORT),
            strokeDash=[4, 2]
        ).encode(
            alt.Y(f'baseline:Q', title='Failure Rate'),
            alt.X()
        ).transform_calculate(
            baseline=f"{US_GOV_FR_MEAN}"
        )
    )
    
    _ = _ + baseline

    plot = _ if plot is None else plot | _

plot = plot.properties(
    title='Failure Rate by Impact Score of Each Website'
)

apply_theme(plot)

## Impact

In [ ]:
_ = (
    alt.Chart(
        df['j_agg'][df['j_agg'].sjr_best_quartile.str.contains('Q')]
    ).mark_tick(
        color='#D20000',
        thickness=2,
        size=40
    ).encode(
        alt.X(f'mean(failure_rate):Q', title='Failure Rate').scale(domain=[0, 0.3], clamp=True).axis(format='%'),
        alt.Y('sjr_best_quartile:N', title=None)
    ).properties(
        title={
            "text": title,
            "color": "grey"
        },
        height=300,
        width=500
    )
)

dist = (
    _.mark_circle(
        color='grey',
        size=20,
        opacity=0.5
    ).encode(
        alt.X(f'failure_rate:Q', title='Failure Rate').scale(domain=[0, .3], clamp=True).axis(format='%'),
        alt.Y(f'sjr_best_quartile:N', title=None),
        alt.Size('total_checks:Q'),
        alt.YOffset('jitter:Q'),
        alt.Tooltip([f'title:N', f'url:N'])
    ).transform_calculate(
        jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
    )
)

baseline = (
    _.mark_rule(
        color='black',
        size=2,
        strokeDash=[4, 2]
    ).encode(
        alt.X(f'baseline:Q', title='Failure Rate'),
        y=alt.Y()
    ).transform_calculate(
        baseline=f"{US_GOV_FR_MEAN}"
    )
)

plot = dist + _ + baseline

plot = plot.properties(
    title="The Failure Rate By Journal Websites (SJR)"
)

apply_theme(plot)

In [ ]:
_ = df['d_agg'].copy()
_ = _[~_.zindex.isnull()]
_.zindex = _.zindex.apply(lambda x: int(x))
df['d_agg'] = _.assign(qaurtile=pd.qcut(_.zindex, 4, ['Q1', 'Q2', 'Q3', 'Q4']))

In [ ]:
_ = (
    alt.Chart(
        df['d_agg'][df['d_agg'].qaurtile.str.contains('Q')]
    ).mark_tick(
        color='#D20000',
        thickness=2,
        size=40
    ).encode(
        alt.X(f'mean(failure_rate):Q', title='Failure Rate').scale(domain=[0, 0.3], clamp=True).axis(format='%'),
        alt.Y('qaurtile:N', title=None)
    ).properties(
        title={
            "text": title,
            "color": "grey"
        },
        height=300,
        width=500
    )
)

dist = (
    _.mark_circle(
        color='grey',
        size=20,
        opacity=0.5
    ).encode(
        alt.X(f'failure_rate:Q', title='Failure Rate').scale(domain=[0, .3], clamp=True).axis(format='%'),
        alt.Y(f'qaurtile:N', title=None),
        alt.Size('total_checks:Q'),
        alt.YOffset('jitter:Q'),
        alt.Tooltip([f'title:N', f'url:N'])
    ).transform_calculate(
        jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
    )
)

baseline = (
    _.mark_rule(
        color='black',
        size=2,
        strokeDash=[4, 2]
    ).encode(
        alt.X(f'baseline:Q', title='Failure Rate'),
        y=alt.Y()
    ).transform_calculate(
        baseline=f"{US_GOV_FR_MEAN}"
    )
)

plot = dist + _ + baseline

plot = plot.properties(
    title="The Failure Rate By Data Portals (Z Index)"
)

apply_theme(plot)

In [ ]:
copy = df['d_agg'].copy()
copy['harvard'] = copy.page_id.apply(lambda x: 'Harvard' if 'harvard' in x else '')

_ = (
    alt.Chart(
        copy
    ).mark_tick(
        color='#D20000',
        thickness=2,
        size=40
    ).encode(
        alt.X(f'mean(failure_rate):Q', title='Failure Rate').scale(domain=[0, 0.3], clamp=True).axis(format='%'),
    ).properties(
        title={
            "text": title,
            "color": "grey"
        },
        height=300,
        width=500
    )
)

dist = (
    _.mark_circle(
        color='grey',
        size=20,
        opacity=0.5
    ).encode(
        alt.X(f'failure_rate:Q', title='Failure Rate').scale(domain=[0, .3], clamp=True).axis(format='%'),
        alt.Size('total_checks:Q'),
        alt.YOffset('jitter:Q'),
        alt.Tooltip([f'title:N', f'url:N'])
    ).transform_calculate(
        jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
    )
)

text = (
    _.mark_bar(
        # color='red',
        # size=20,
        # opacity=0.5
    ).encode(
        alt.X(f'failure_rate:Q', title='Failure Rate').scale(domain=[0, .3], clamp=True).axis(format='%'),
        alt.Y('title:N'),
        # alt.YOffset('jitter:Q'),
        # alt.Tooltip([f'title:N', f'url:N'])
    ).transform_filter(
        (datum.harvard == 'Harvard')
    )
)

baseline = (
    _.mark_rule(
        color='black',
        size=2,
        strokeDash=[4, 2]
    ).encode(
        alt.X(f'baseline:Q', title='Failure Rate'),
        y=alt.Y()
    ).transform_calculate(
        baseline=f"{US_GOV_FR_MEAN}"
    )
)

plot = dist + _ + baseline & text

plot = plot.properties(
    title="The Failure Rates of Our Websites"
)

apply_theme(plot)
# df['d_agg'][df['d_agg'].short_name.str.contains('4D')].short_name

In [ ]:
copy = df['j_agg'].copy()
copy['informatics'] = copy.page_id.apply(lambda x: 'Informatics' if 'informatics' in x.lower() else '')

_ = (
    alt.Chart(
        copy
    ).mark_tick(
        color='#D20000',
        thickness=2,
        size=40
    ).encode(
        alt.X(f'mean(failure_rate):Q', title='Failure Rate').scale(domain=[0, 0.3], clamp=True).axis(format='%'),
    ).properties(
        title={
            "text": title,
            "color": "grey"
        },
        height=300,
        width=500
    )
)

dist = (
    _.mark_circle(
        color='grey',
        size=20,
        opacity=0.5
    ).encode(
        alt.X(f'failure_rate:Q', title='Failure Rate').scale(domain=[0, .3], clamp=True).axis(format='%'),
        alt.Size('total_checks:Q'),
        alt.YOffset('jitter:Q'),
        alt.Tooltip([f'title:N', f'url:N'])
    ).transform_calculate(
        jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
    )
)

informatics = (
    _.mark_circle(
        color='red',
        size=20,
        opacity=0.5
    ).encode(
        alt.X(f'failure_rate:Q', title='Failure Rate').scale(domain=[0, .3], clamp=True).axis(format='%'),
        alt.Size('total_checks:Q'),
        alt.YOffset('jitter:Q'),
        alt.Tooltip([f'title:N', f'url:N'])
        # alt.YOffset('jitter:Q'),
        # alt.Tooltip([f'title:N', f'url:N'])
    ).transform_filter(
        (datum.informatics == 'Informatics')
    )
)

baseline = (
    _.mark_rule(
        color='black',
        size=2,
        strokeDash=[4, 2]
    ).encode(
        alt.X(f'baseline:Q', title='Failure Rate'),
        y=alt.Y()
    ).transform_calculate(
        baseline=f"{US_GOV_FR_MEAN}"
    )
)

plot = dist + _ + baseline + informatics

plot = plot.properties(
    title="The Failure Rates of Our Websites"
)

apply_theme(plot)
# df['d_agg'][df['d_agg'].short_name.str.contains('4D')].short_name

## By Group?

In [ ]:
plot = None
for (id, category, title) in A11Y_CATEGORIES:
    if id == 'g':
        continue
    
    impact_field = 'h_index' if id == 'j' else 'citation'
    gropu_field = 'publisher' if id == 'j' else 'host_institution'

    _ = (
        alt.Chart(
            df[id + '_agg']
        ).mark_point(
            # filled=True,
            opacity=0.3,
            color='#56B4E9'
        ).encode(
            alt.X(f'mean({impact_field}):Q', title=impact_field.capitalize()).scale(type='symlog'),
            alt.Y('mean(failure_rate):Q', title='Failure Rate').scale(domain=[0, 1]),
            alt.Color(f'{gropu_field}:N').scale(range=['#56B4E9']),
            alt.Tooltip([f'{gropu_field}:N'])
            # alt.Tooltip([f'{name_field}:N', f'{url_field}:N'])
        ).properties(
            title={
                'text': title,
                'color': 'grey'
            },
            width=400,
            height=300
        )
    )

    baseline = (
        _.mark_rule(
            color='black',
            size=2,
            # size=500 / len(COUNTRY_SORT),
            strokeDash=[4, 2]
        ).encode(
            alt.Y(f'baseline:Q', title='Failure Rate'),
            alt.X(),
            alt.Color()
        ).transform_calculate(
            baseline=f"{US_GOV_FR_MEAN}"
        )
    )
    
    _ = _ + baseline

    plot = _ if plot is None else plot | _

plot = plot.properties(
    title='Failure Rate by Impact Score of Each Website'
)

apply_theme(plot)

## What are the Issues checked?

In [ ]:
plots = []
for (id, category, title) in A11Y_CATEGORIES:
    _ = (
        alt.Chart(
            df[id]#[df[id].issue_id != 'region']
        ).mark_bar(
            color='#56B4E9'
        ).encode(
            alt.Y('issue_id:N'),
            alt.X(f'sum(total_checks):Q', title='The Number of Items Checked'),#.scale(type='log'),
            alt.Tooltip([f'title:N', f'url:N'])
        ).properties(
            title={
                'text': title,
                'color': 'grey'
            },
            width=400,
            height=1000
        )
    )
    plots.append(_)

# plot = plot.properties(
#     title='The Number of Items Checked vs. Failure Rate'
# )
    
plot = alt.hconcat(*plots).resolve_scale(y='independent')

apply_theme(plot)

In [ ]:
merged = pd.concat([df['dp'], df['jp']])
merged['issue_exist'] = merged.violations.apply(lambda x: 1 if x > 0 else 0)
merged

In [ ]:
_ = (
    alt.Chart(
        merged
    ).mark_tick(
        color='#D20000',
        thickness=2,
        # size=100
    ).encode(
        alt.X(f'sum(issue_exist):Q', title='The Number of Websites Having Issues'),
        alt.Y('issue_id:N', title=None, sort='x')
    ).properties(
        title={
            "text": title,
            "color": "grey"
        },
        height=900,
        width=500
    )
)

# dist = (
#     _.mark_circle(
#         color='grey',
#         size=20,
#         opacity=0.5
#     ).encode(
#         alt.X(f'failure_rate:Q', title='Failure Rate').scale(domain=[0, .5], clamp=True).axis(format='%'),
#         alt.Y(f'issue_id:N', title=None),
#         # alt.Size('total_checks:Q'),
#         alt.YOffset('jitter:Q'),
#         # alt.Tooltip([f'{name_field}:N', f'{url_field}:N'])
#     ).transform_calculate(
#         jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
#     )
# )

# baseline = (
#     _.mark_rule(
#         color='black',
#         size=2,
#         strokeDash=[4, 2]
#     ).encode(
#         alt.X(f'baseline:Q', title='Failure Rate'),
#         y=alt.Y()
#     ).transform_calculate(
#         baseline=f"{US_GOV_FR_MEAN}"
#     )
# )

# plot = dist + _ + baseline
# plot = _ + baseline
plot = _

plot = plot.properties(
    title="The Failure Rate of Biomedical Websites By Domain"
)

apply_theme(plot)

## By Country

In [ ]:
plot = None
for (id, category, title) in A11Y_CATEGORIES:
    if id == 'g':
        continue

    # Select Countries with more than N pages
    CUT = 10
    _ = df[id + '_agg'].copy()
    df_count = _.country.value_counts().reset_index().sort_values(by='count', ascending=False)
    df_count = df_count[df_count['count'] > CUT]
    COUNTRY_FILTER = df_count.country.tolist()

    # Sort
    df_fr = _[_.country.isin(COUNTRY_FILTER)][['country', 'failure_rate']].groupby(['country']).mean().reset_index().sort_values(by='failure_rate', ascending=False)
    COUNTRY_SORT = df_fr.country.tolist()

    # Add "Other"
    _['country_with_other'] = _.country.apply(lambda x: x if x in COUNTRY_FILTER else 'Other')

    _ = (
        alt.Chart(
            _#[_.country.isin(COUNTRY_FILTER)]
        ).mark_tick(
            # opacity=0.01
            color='#D20000',
            thickness=2,
            size=20
        ).encode(
            alt.X(f'mean(failure_rate):Q', title='Failure Rate').scale(domain=[0, 0.5], clamp=True).axis(format='%'),
            alt.Y('country_with_other:N', title=None, sort=COUNTRY_SORT)
        ).properties(
            title={
                "text": title,
                "color": "grey"
            },
            height=1000,
            width=500
        )
    )

    dist = (
        _.mark_circle(
            color='grey',
            # size=10,
            opacity=0.5
        ).encode(
            alt.X(f'failure_rate:Q', title='Failure Rate').scale(domain=[0, 0.5], clamp=True).axis(format='%'),
            alt.Y('country_with_other:N', title=None, sort=COUNTRY_SORT),
            alt.Size('total_checks:Q').scale(type='linear', range=[4, 300]),
            alt.YOffset('jitter:Q'),
            alt.Tooltip(['title:N', 'url:N'])
        ).transform_calculate(
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
    )

    baseline = (
        _.mark_rule(
            color='black',
            size=2,
            # size=500 / len(COUNTRY_SORT),
            strokeDash=[4, 2]
        ).encode(
            alt.X(f'baseline:Q', title='Failure Rate'),
            y=alt.Y()
        ).transform_calculate(
            baseline=f"{US_GOV_FR_MEAN}"
        )
    )
    
    _ = (dist + _ + baseline).resolve_axis(y='shared')

    plot = _ if plot is None else plot | _

plot = plot.resolve_axis(y='independent')

plot = plot.properties(
    title={
        'text': 'The Failure Rate of Webpages by Country Compared to Baseline',
        'subtitle': [
            f'* Only countries with more than {CUT} pages are shown',
            '* US Government websites are used as the baseline (dashed line)'
        ],
        'subtitleColor': 'grey'
    }
)

apply_theme(plot)

## By Continent

In [ ]:
plot = None
for (id, category, title) in A11Y_CATEGORIES:
    if id == 'g':
        continue

    # Select Countries with more than N pages
    # CUT = 10
    _ = df[id + '_agg'].copy()
    # df_count = _.country.value_counts().reset_index().sort_values(by='count', ascending=False)
    # df_count = df_count[df_count['count'] > CUT]
    # COUNTRY_FILTER = df_count.country.tolist()

    # Sort
    df_fr = _[['continent', 'failure_rate']].groupby(['continent']).mean().reset_index().sort_values(by='failure_rate', ascending=False)
    SORT = df_fr.continent.tolist()

    # Add "Other"
    # _['country_with_other'] = _.country.apply(lambda x: x if x in COUNTRY_FILTER else 'Other')

    _ = (
        alt.Chart(
            _
        ).mark_tick(
            # opacity=0.01
            color='#D20000',
            thickness=2,
            size=20
        ).encode(
            alt.X(f'mean(failure_rate):Q', title='Failure Rate').scale(domain=[0, 0.5], clamp=True).axis(format='%'),
            alt.Y('continent:N', title=None, sort=SORT)
        ).properties(
            title={
                "text": title,
                "color": "grey"
            },
            height=500,
            width=500
        )
    )

    dist = (
        _.mark_circle(
            color='grey',
            # size=10,
            opacity=0.5
        ).encode(
            alt.X(f'failure_rate:Q', title='Failure Rate').scale(domain=[0, 0.5], clamp=True).axis(format='%'),
            alt.Y('continent:N', title=None, sort=SORT),
            alt.Size('total_checks:Q').scale(type='linear', range=[4, 300]),
            alt.YOffset('jitter:Q'),
            alt.Tooltip(['title:N', 'url:N'])
        ).transform_calculate(
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
    )

    baseline = (
        _.mark_rule(
            color='black',
            size=2,
            # size=500 / len(COUNTRY_SORT),
            strokeDash=[4, 2]
        ).encode(
            alt.X(f'baseline:Q', title='Failure Rate'),
            y=alt.Y()
        ).transform_calculate(
            baseline=f"{US_GOV_FR_MEAN}"
        )
    )
    
    _ = (dist + _ + baseline).resolve_axis(y='shared')

    plot = _ if plot is None else plot | _

plot = plot.resolve_axis(y='independent')

plot = plot.properties(
    title={
        'text': 'The Failure Rate by Continent',
        'subtitle': [
            f'* Only countries with more than {CUT} pages are shown',
            '* US Government websites are used as the baseline (dashed line)'
        ],
        'subtitleColor': 'grey'
    }
)

apply_theme(plot)

## By Organizations

In [ ]:
plot = None
for (id, category, title) in A11Y_CATEGORIES:
    if id == 'g':
        continue
    
    category_field = 'publisher' if id == 'j' else 'host_institution'
    
    _ = df[id + '_agg'].copy()

    # if id == 'd':
    #     _[category_field] = _[category_field].apply(lambda x: 'University' if 'university' in str(x).lower() else x)
    
    # Select Countries with more than 10 pages
    CUT = 10
    df_count = _[category_field].value_counts().reset_index().sort_values(by='count', ascending=False)
    df_count = df_count[df_count['count'] > CUT]
    FILTER = df_count[category_field].tolist()

    # Sort
    df_fr = _[_[category_field].isin(FILTER)][[category_field, 'failure_rate']].groupby([category_field]).mean().reset_index().sort_values(by='failure_rate', ascending=False)
    SORT = df_fr[category_field].tolist()

    _ = (
        alt.Chart(
            _[_[category_field].isin(FILTER)]
        ).mark_tick(
            # opacity=0.01
            color='#D20000',
            thickness=2,
            # size=500 / len(COUNTRY_SORT),
        ).encode(
            alt.X(f'mean(failure_rate):Q', title='Failure Rate').scale(domain=[0, 0.5], clamp=True).axis(format='%'),
            alt.Y(f'{category_field}:N', sort=SORT, title=None)
        ).properties(
            title={
                "text": title,
                "color": "grey"
            },
            # height=500,
            width=500
        )
    )

    dist = (
        _.mark_circle(
            color='grey',
            size=20,
            opacity=0.5
        ).encode(
            alt.X(f'failure_rate:Q', title='Failure Rate').scale(domain=[0, 0.5], clamp=True).axis(format='%'),
            alt.Y(f'{category_field}:N', sort=SORT, title=None),
            alt.Size('total_checks:Q').scale(type='linear', range=[4, 300]),
            alt.YOffset('jitter:Q'),
            alt.Tooltip([f'title:N', f'url:N'])
        ).transform_calculate(
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
    )

    baseline = (
        _.mark_rule(
            color='black',
            size=2,
            # size=500 / len(COUNTRY_SORT),
            strokeDash=[4, 2]
        ).encode(
            alt.X(f'baseline:Q', title='Failure Rate'),
            y=alt.Y()
        ).transform_calculate(
            baseline=f"{US_GOV_FR_MEAN}"
        )
    )
    
    _ = dist + _ + baseline

    plot = _ if plot is None else plot | _

plot = plot.resolve_axis(y='independent')

plot = plot.properties(
    title={
        'text': 'The Failure Rate of Webpages by Organization Compared to Baseline',
        'subtitle': [
            f'* Only publishers/institutions with {CUT} pages or more are shown',
            '* US Government websites are used as the baseline (Dashed Line)'
        ],
        'subtitleColor': 'grey'
    }
)

apply_theme(plot)

In [ ]:
_ = df['j_agg'].copy()
plot = alt.Chart(
    _[_.publisher == 'Nature Publishing Group']
).mark_bar(
    
).encode(
    alt.X('mean(failure_rate):Q'),
    alt.Y('title:N', sort='x')
)
apply_theme(plot)
# _[_.title == 'Nature Communications']

In [ ]:
_ = df['dpg']
_[_.host_institution == 'Zhejiang University'].sort_values(by='failure_rate', ascending=False)[['short_name', 'url']]
# _[_.host_institution == 'European Bioinformatics Institute'].sort_values(by='failure_rate', ascending=True).short_name
# _[_.host_institution == 'National Institutes of Halth'].sort_values(by='failure_rate', ascending=True).short_name

In [ ]:
_ = df['jpg']
_[_.publisher == 'Nature Publishing Group'].sort_values(by='failure_rate', ascending=True)[['title', 'url']].head(40)
# _[_.publisher == 'Springer'].sort_values(by='failure_rate', ascending=True)[['title', 'url']].head(50)
# _[_.publisher == 'BioMed Central Ltd.'].sort_values(by='failure_rate', ascending=True)[['title', 'url']].head(50)
# _[_.title == 'Nature Communications']

## How does it look with NIH websites?

In [ ]:
name_field = 'title'
category_field = 'host_institution'

_ = df['d_agg'].copy()

_ = _[_.host_institution.isin(NIH_INSTS)]

# Select Countries with more than 10 pages
CUT = 1
df_count = _[category_field].value_counts().reset_index().sort_values(by='count', ascending=False)
df_count = df_count[df_count['count'] > CUT]
FILTER = df_count[category_field].tolist()

# Sort
df_fr = _[_[category_field].isin(FILTER)][[category_field, 'failure_rate']].groupby([category_field]).mean().reset_index().sort_values(by='failure_rate', ascending=False)
SORT = df_fr[category_field].tolist()

_ = (
    alt.Chart(
        _[_[category_field].isin(FILTER)]
    ).mark_tick(
        # opacity=0.01
        color='#D20000',
        thickness=2,
        # size=500 / len(COUNTRY_SORT),
    ).encode(
        alt.X(f'mean(failure_rate):Q', title='Failure Rate').scale(domain=[0, 1]).axis(format='%'),
        alt.Y(f'{category_field}:N', sort=SORT, title=None)
    ).properties(
        title={
            "text": title,
            "color": "grey"
        },
        # height=500,
        width=500
    )
)

dist = (
    _.mark_circle(
        color='grey',
        size=20,
        opacity=0.5
    ).encode(
        alt.X(f'failure_rate:Q', title='Failure Rate').scale(domain=[0, 1]).axis(format='%'),
        alt.Y(f'{category_field}:N', sort=SORT, title=None),
        alt.YOffset('jitter:Q'),
        alt.Tooltip([f'title:N', f'url:N'])
    ).transform_calculate(
        jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
    )
)

baseline = (
    _.mark_rule(
        color='black',
        size=2,
        # size=500 / len(COUNTRY_SORT),
        strokeDash=[4, 2]
    ).encode(
        alt.X(f'baseline:Q', title='Failure Rate'),
        y=alt.Y()
    ).transform_calculate(
        baseline=f"{US_GOV_FR_MEAN}"
    )
)

_ = dist + _ + baseline

plot = _

plot = plot.properties(
    title={
        'text': 'The Failure Rate of NIH Data Portals Compared to Baseline',
        'subtitle': [
            '* US Government websites are used as the baseline (Dashed Line)'
        ],
        'subtitleColor': 'grey'
    }
)

apply_theme(plot)

# What are common issues found from NIH data portals?

In [ ]:
alt.Chart(
    df['dp'][df['dp'].host_institution == 'National Institutes of Health']
).mark_bar(
    color='#56B4E9'
).encode(
    alt.Y('issue_id:N'),
    alt.X(f'mean(failure_rate):Q', title='The Number of Items Checked'),#.scale(type='log'),
).properties(
    title={
        'text': title,
        'color': 'grey'
    },
    width=400,
    height=1000
)

# .gov Data Portals vs. Other Data Portals

In [ ]:
id = 'd'
_df = df[id + '_agg'][df[id + '_agg'].failure_rate > 0].copy()
dotgov = _df[_df.url.str.contains('.gov')]
all = (
    alt.Chart(
        _df
    ).mark_bar(
        color='#56B4E9'
    ).encode(
        alt.X(f'failure_rate:Q', title='Failure Rate').bin(extent=[0, 1], step=0.01).scale(domain=[0, 0.5], clamp=True).axis(format='%'),
        alt.Y('count()', title="The Number of Websites").scale(type='linear'),
    ).properties(
        title={
            "text": title,
            "color": "grey"
        },
        height=300,
        width=500
    )
)

onlydotgov = (
    alt.Chart(
        dotgov
    ).mark_bar(
        # opacity=0.01
        color='#0072B2'
    ).encode(
        alt.X(f'failure_rate:Q', title='Failure Rate').bin(extent=[0, 1], step=0.01).scale(domain=[0, 0.5], clamp=True).axis(format='%'),
        alt.Y('count()', title="The Number of Websites").scale(type='linear'),
    ).properties(
        title={
            "text": title,
            "color": "grey"
        },
        height=300,
        width=500
    )
)

baseline = (
    all.mark_rule(
        color='black',
        size=2,
        # size=500 / len(COUNTRY_SORT),
        strokeDash=[4, 2]
    ).encode(
        alt.X(f'baseline:Q', title='Failure Rate'),
        y=alt.Y()
    ).transform_calculate(
        baseline=f"{US_GOV_FR_MEAN}"
    )
)

_ = all + onlydotgov + baseline
_
_ = _.properties(
    title={
        'text': 'The Distribution of Failure Rate of Data Portals',
        'subtitle': [
            '* Dashed line represents the average failure rate of US government websites',
            '* Dark blue bars represent .gov websites'
        ],
        'subtitleColor': 'grey'
    }
)
apply_theme(_)

# .gov vs. All Others

In [ ]:
df['d_agg']['resource_type'] = 'Data Portals'
df['j_agg']['resource_type'] = 'Journal Portals'
df['g_agg']['resource_type'] = 'Government Websites'
merged = pd.concat([df['d_agg'], df['j_agg']]) # , df['gog']])
merged['is_dot_gov'] = merged['url'].apply(lambda x: 'Government Biomedical Websites' if '.gov' in x.lower() else 'Non-government Biomedical Websites')
df['g_agg']['is_dot_gov'] = 'Government Non-biomedical Websites'
merged = pd.concat([merged, df['g_agg']])

In [ ]:
_ = (
    alt.Chart(
        merged
    ).mark_tick(
        color='#D20000',
        thickness=2,
        size=100
    ).encode(
        alt.X(f'mean(failure_rate):Q', title='Failure Rate').scale(domain=[0, 0.5], clamp=True).axis(format='%'),
        alt.Y('is_dot_gov:N', title=None)
    ).properties(
        title={
            "text": title,
            "color": "grey"
        },
        height=300,
        width=500
    )
)

dist = (
    _.mark_circle(
        color='grey',
        size=20,
        opacity=0.5
    ).encode(
        alt.X(f'failure_rate:Q', title='Failure Rate').scale(domain=[0, .5], clamp=True).axis(format='%'),
        alt.Y(f'is_dot_gov:N', title=None),
        alt.YOffset('jitter:Q'),
        alt.Tooltip([f'title:N', f'url:N'])
    ).transform_calculate(
        jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
    )
)

baseline = (
    _.mark_rule(
        color='black',
        size=2,
        strokeDash=[4, 2]
    ).encode(
        alt.X(f'baseline:Q', title='Failure Rate'),
        y=alt.Y()
    ).transform_calculate(
        baseline=f"{US_GOV_FR_MEAN}"
    )
)

plot = dist + _ + baseline

plot = plot.properties(
    title="The Failure Rate of Biomedical Websites By Domain"
)

apply_theme(plot)

In [ ]:
_ = (
    alt.Chart(
        merged
    ).mark_tick(
        color='#D20000',
        thickness=2,
        size=100
    ).encode(
        alt.X(f'mean(failure_rate):Q', title='Failure Rate').scale(domain=[0, 0.5], clamp=True).axis(format='%'),
        alt.Y('is_dot_gov:N', title=None)
    ).properties(
        title={
            "text": title,
            "color": "grey"
        },
        height=300,
        width=500
    )
)

dist = (
    _.mark_circle(
        color='grey',
        size=20,
        opacity=0.5
    ).encode(
        alt.X(f'failure_rate:Q', title='Failure Rate').scale(domain=[0, .5], clamp=True).axis(format='%'),
        alt.Y(f'is_dot_gov:N', title=None),
        alt.Size('total_checks:Q'),
        alt.YOffset('jitter:Q'),
        alt.Tooltip([f'title:N', f'url:N'])
    ).transform_calculate(
        jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
    )
)

baseline = (
    _.mark_rule(
        color='black',
        size=2,
        strokeDash=[4, 2]
    ).encode(
        alt.X(f'baseline:Q', title='Failure Rate'),
        y=alt.Y()
    ).transform_calculate(
        baseline=f"{US_GOV_FR_MEAN}"
    )
)

plot = dist + _ + baseline

plot = plot.properties(
    title="The Failure Rate of Biomedical Websites By Domain"
)

apply_theme(plot)

In [ ]:
_ = (
    alt.Chart(
        merged[merged.resource_type == 'Data Portals']
    ).mark_tick(
        color='#D20000',
        thickness=2,
        size=100
    ).encode(
        alt.X(f'mean(failure_rate):Q', title='Failure Rate').scale(domain=[0, 0.5], clamp=True).axis(format='%'),
        alt.Y('is_dot_gov:N', title=None)
    ).properties(
        title={
            "text": title,
            "color": "grey"
        },
        height=300,
        width=500
    )
)

dist = (
    _.mark_circle(
        color='grey',
        size=20,
        opacity=0.5
    ).encode(
        alt.X(f'failure_rate:Q', title='Failure Rate').scale(domain=[0, .5], clamp=True).axis(format='%'),
        alt.Y(f'is_dot_gov:N', title=None),
        alt.YOffset('jitter:Q'),
        alt.Tooltip([f'{name_field}:N', f'{url_field}:N'])
    ).transform_calculate(
        jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
    )
)

baseline = (
    _.mark_rule(
        color='black',
        size=2,
        strokeDash=[4, 2]
    ).encode(
        alt.X(f'baseline:Q', title='Failure Rate'),
        y=alt.Y()
    ).transform_calculate(
        baseline=f"{US_GOV_FR_MEAN}"
    )
)

plot = dist + _ + baseline

plot = plot.properties(
    title="The Failure Rate of Data Portals By Domain"
)

apply_theme(plot)

# Within .gov, Biomedical vs. Others

In [ ]:
_ = (
    alt.Chart(
        merged[merged.is_dot_gov == '.gov']
    ).mark_tick(
        color='#D20000',
        thickness=2,
        size=100
    ).encode(
        alt.X(f'mean(failure_rate):Q', title='Failure Rate').scale(domain=[0, 0.5], clamp=True).axis(format='%'),
        alt.Y('resource_type:N', title=None)
    ).properties(
        title={
            "text": title,
            "color": "grey"
        },
        height=300,
        width=500
    )
)

dist = (
    _.mark_circle(
        color='grey',
        size=20,
        opacity=0.5
    ).encode(
        alt.X(f'failure_rate:Q', title='Failure Rate').scale(domain=[0, .5], clamp=True).axis(format='%'),
        alt.Y(f'resource_type:N', title=None),
        alt.YOffset('jitter:Q'),
        alt.Tooltip([f'short_name:N', f'url:N'])
    ).transform_calculate(
        jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
    )
)

baseline = (
    _.mark_rule(
        color='black',
        size=2,
        strokeDash=[4, 2]
    ).encode(
        alt.X(f'baseline:Q', title='Failure Rate'),
        y=alt.Y()
    ).transform_calculate(
        baseline=f"{US_GOV_FR_MEAN}"
    )
)

plot = dist + baseline + _

plot = plot.properties(
    title="The Failure Rate of .gov Websites"
)

apply_theme(plot)

# Curious about the relation with founded_year for data portals

In [ ]:
plot = alt.Chart(
    # df['dpg']
    # or
    df['d_agg']#[df['d_agg']#.host_institution == 'National Institutes of Health']
).mark_bar(
    # opacity=0.01
).encode(
    alt.Y(f'mean(failure_rate):Q'),
    alt.X('founded_year:O').axis(labelAngle=-90),
    # alt.Size('mean(total_checks):Q')
).properties(
    width=500
)

_ = (
    alt.Chart(
        merged[~merged.founded_year.isnull()]
    ).mark_line(
        # color='#D20000',
        thickness=2,
        point=True
        # size=100
    ).encode(
        alt.Y(f'mean(failure_rate):Q', title='Failure Rate').scale(domain=[0, 0.2], clamp=True).axis(format='%'),
        alt.X('founded_year:O', title=None).axis(labelAngle=-90),
    ).properties(
        title={
            "text": title,
            "color": "grey"
        },
        height=600,
        width=500
    )
)

dist = (
    _.mark_circle(
        color='grey',
        size=20,
        opacity=0.5
    ).encode(
        alt.Y(f'failure_rate:Q', title='Failure Rate').scale(domain=[0, .2], clamp=True).axis(format='%'),
        alt.X('founded_year:O', title=None).axis(labelAngle=-90),
        alt.Size('total_checks:Q'),
        alt.XOffset('jitter:Q'),
        alt.Tooltip([f'title:N', f'url:N'])
    ).transform_calculate(
        jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
    )
)

baseline = (
    _.mark_rule(
        color='black',
        size=2,
        strokeDash=[4, 2]
    ).encode(
        alt.Y(f'baseline:Q', title='Failure Rate'),
        x=alt.X()
    ).transform_calculate(
        baseline=f"{US_GOV_FR_MEAN}"
    )
)

plot = _ + dist + baseline

plot = plot.properties(
    title="The Failure Rate of Data Portals By Founded Year"
)

apply_theme(plot)

# How about the alt texts?

In [ ]:
US_GOV_ALT_FR_MEAN = df['goag'].failure_rate.mean()
US_GOV_ALT_FR_MEAN

In [ ]:
plot = None
for (id, category, title, name_field, url_field) in A11Y_CATEGORIES:
    _ = (
        alt.Chart(
            df[id + 'ag'][df[id + 'ag'].failure_rate >= 0]
        ).mark_bar(
            # opacity=0.01
            color='#56B4E9'
        ).encode(
            alt.X(f'failure_rate:Q', title='Failure Rate').bin(extent=[0, 1], step=0.01).scale(domain=[0, 1]).axis(format='%'),
            alt.Y('count()').scale(type='log'),
        ).properties(
            title={
                "text": title,
                "color": "grey"
            },
            height=300,
            width=500
        )
    )

    baseline = (
        _.mark_rule(
            color='black',
            size=2,
            # size=500 / len(COUNTRY_SORT),
            strokeDash=[4, 2]
        ).encode(
            alt.X(f'baseline:Q', title='Failure Rate'),
            alt.Y(),
            alt.Color()
        ).transform_calculate(
            baseline=f"{US_GOV_ALT_FR_MEAN}"
        )
    )
    
    _ = _ + baseline

    plot = _ if plot is None else plot | _

plot = plot.properties(
    title='The Distribution of Failure Rate of Webpages'
)

apply_theme(plot)

In [ ]:
plot = None
for (id, category, title, name_field, url_field) in A11Y_CATEGORIES:
    if id == 'go':
        continue
    
    category_field = 'publisher' if id == 'jp' else 'host_institution'
    
    _ = df[id + 'ag'].copy()

    # Select Countries with more than 10 pages
    CUT = 10
    df_count = _[category_field].value_counts().reset_index().sort_values(by='count', ascending=False)
    df_count = df_count[df_count['count'] > CUT]
    FILTER = df_count[category_field].tolist()

    # Sort
    df_fr = _[_[category_field].isin(FILTER)][[category_field, 'failure_rate']].groupby([category_field]).mean().reset_index().sort_values(by='failure_rate', ascending=False)
    SORT = df_fr[category_field].tolist()

    _ = (
        alt.Chart(
            _[_[category_field].isin(FILTER)]
        ).mark_tick(
            # opacity=0.01
            color='#D20000',
            thickness=2,
            # size=500 / len(COUNTRY_SORT),
        ).encode(
            alt.X(f'mean(failure_rate):Q', title='Failure Rate').scale(domain=[0, 1]).axis(format='%'),
            alt.Y(f'{category_field}:N', sort=SORT, title=None)
        ).properties(
            title={
                "text": title,
                "color": "grey"
            },
            # height=500,
            width=500
        )
    )

    dist = (
        _.mark_circle(
            color='grey',
            size=20,
            opacity=0.5
        ).encode(
            alt.X(f'failure_rate:Q', title='Failure Rate').scale(domain=[0, 1]).axis(format='%'),
            alt.Y(f'{category_field}:N', sort=SORT, title=None),
            alt.YOffset('jitter:Q'),
            alt.Tooltip([f'{name_field}:N', f'{url_field}:N'])
        ).transform_calculate(
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
    )

    baseline = (
        _.mark_rule(
            color='black',
            size=2,
            # size=500 / len(COUNTRY_SORT),
            strokeDash=[4, 2]
        ).encode(
            alt.X(f'baseline:Q', title='Failure Rate'),
            y=alt.Y()
        ).transform_calculate(
            baseline=f"{US_GOV_FR_MEAN}"
        )
    )
    
    _ = dist + _ + baseline

    plot = _ if plot is None else plot | _

plot = plot.resolve_axis(y='independent')

plot = plot.properties(
    title={
        'text': 'The Failure Rate of Webpages by Organization Compared to Baseline',
        'subtitle': [
            f'* Only publishers/institutions with {CUT} pages or more are shown',
            '* US Government websites are used as the baseline (Dashed Line)'
        ],
        'subtitleColor': 'grey'
    }
)

apply_theme(plot)

In [ ]:
plot = None
for (id, category, title, name_field, url_field) in A11Y_CATEGORIES:
    if id == 'go':
        continue

    # Select Countries with more than 10 pages
    CUT = 10
    _ = df[id + 'ag'].copy()
    df_count = _.country.value_counts().reset_index().sort_values(by='count', ascending=False)
    df_count = df_count[df_count['count'] > CUT]
    COUNTRY_FILTER = df_count.country.tolist()

    # Sort
    df_fr = _[_.country.isin(COUNTRY_FILTER)][['country', 'failure_rate']].groupby(['country']).mean().reset_index().sort_values(by='failure_rate', ascending=False)
    COUNTRY_SORT = df_fr.country.tolist()

    _ = (
        alt.Chart(
            _[_.country.isin(COUNTRY_FILTER)]
        ).mark_tick(
            # opacity=0.01
            color='#D20000',
            thickness=2,
            size=22
        ).encode(
            alt.X(f'mean(failure_rate):Q', title='Failure Rate').scale(domain=[0, 1]).axis(format='%'),
            alt.Y('country:N', sort=COUNTRY_SORT)
        ).properties(
            title={
                "text": title,
                "color": "grey"
            },
            height=500,
            width=500
        )
    )

    dist = (
        _.mark_circle(
            color='grey',
            size=10,
            opacity=0.5
        ).encode(
            alt.X(f'failure_rate:Q', title='Failure Rate').scale(domain=[0, 1]).axis(format='%'),
            alt.Y('country:N', sort=COUNTRY_SORT),
            alt.YOffset('jitter:Q')
        ).transform_calculate(
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
    )

    baseline = (
        _.mark_rule(
            color='black',
            size=2,
            # size=500 / len(COUNTRY_SORT),
            strokeDash=[4, 2]
        ).encode(
            alt.X(f'baseline:Q', title='Failure Rate'),
            y=alt.Y()
        ).transform_calculate(
            baseline=f"{US_GOV_FR_MEAN}"
        )
    )
    
    _ = dist + _ + baseline

    plot = _ if plot is None else plot | _

plot = plot.resolve_axis(y='independent')

plot = plot.properties(
    title={
        'text': 'The Failure Rate of Webpages by Country Compared to Baseline',
        'subtitle': [
            f'* Only countries with more than {CUT} pages are shown',
            '* US Government websites are used as the baseline (dashed line)'
        ],
        'subtitleColor': 'grey'
    }
)

apply_theme(plot)

In [ ]:
plots = []
for (id, category, title, name_field, url_field) in A11Y_CATEGORIES:
    _ = (
        alt.Chart(
            df[id + 'g']
        ).mark_bar(
            # opacity=0.01
        ).encode(
            alt.X(f'mean(total_checks):Q'),
            alt.Y('country:N', sort='-x'),
            alt.Color('count()'),
            alt.Tooltip(['count()'])
        ).properties(
            title='Countries with more than 10 sites',
            width=500
        )
    )
    plots.append(_)
apply_theme(alt.vconcat(*plots))

In [ ]:
alt.Chart(
    _df[_df.country.isin(COUNTRIES)]
).mark_bar(
    # opacity=0.01
).encode(
    alt.X(f'mean(A3):Q'),
    alt.Y('country:N', sort='-x'),
    alt.Color('count()'),
    alt.Tooltip(['count()'])
).properties(
    title='Countries with more than 10 sites',
    width=500
)

In [ ]:
alt.Chart(
    _df[_df.country.isin(COUNTRIES)]
).mark_bar(
    # opacity=0.01
).encode(
    alt.X(f'mean(violation_ratio):Q', title='The ratio of violations among all potential violations').axis(format='.1%').scale(alt.Scale(domain=(0, 1))),
    alt.Y('country:N', sort='-x'),
    alt.Color('count()'),
    alt.Tooltip(['count()'])
).properties(
    title='Countries with more than 10 sites',
    width=500
)

In [ ]:
df_host_institution_count = _df.host_institution.value_counts().reset_index().sort_values(by='count', ascending=False)
df_host_institution_count = df_host_institution_count[df_host_institution_count['count'] > 10]
INSTS = df_host_institution_count.host_institution.tolist()
INSTS

In [ ]:
alt.Chart(
    _df[_df.host_institution.isin(INSTS)]
).mark_bar(
    # opacity=0.01
).encode(
    alt.X(f'mean(Np):Q'),
    alt.Y('host_institution:N', sort='-x'),
    alt.Color('count()'),
    alt.Tooltip(['count()'])
).properties(
    title='host_institution with more than 10 sites',
    width=500
)